In [1]:
import numpy as np
import pandas as pd

In [2]:
df_1 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_1.csv")
df_2 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_2.csv")
df_3 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_3.csv")
df = pd.concat([df_1, df_2, df_3], ignore_index=True)

/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_66597/3819499800.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_2.csv")
/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_66597/3819499800.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_3 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_3.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204874 entries, 0 to 2204873
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 218.7+ MB


In [4]:
df["rideable_type"].value_counts()

rideable_type
electric_bike    1364292
classic_bike      840582
Name: count, dtype: int64

In [5]:
df["member_casual"].value_counts()

member_casual
member    1869769
casual     335105
Name: count, dtype: int64

In [9]:
df["start_station_id"].nunique()

4318

In [10]:
df["start_station_name"].nunique()

2134

In [13]:
df.groupby("start_station_name")[["start_lat", "start_lng"]].var()

,start_lat,start_lng
start_station_name,,
1 Ave & E 110 St,0.0,0.0
1 Ave & E 16 St,0.0,0.0
1 Ave & E 18 St,0.0,0.0
1 Ave & E 30 St,0.0,0.0
1 Ave & E 39 St,0.0,0.0
...,...,...
Wyckoff Ave & Stanhope St,0.0,0.0
Wyckoff St & 3 Ave,0.0,0.0
Wythe Ave & Metropolitan Ave,0.0,0.0


In [14]:
mapping = (
    df.groupby(["start_station_name", "start_station_id"])
    .size()
    .reset_index()
)

In [15]:
mapping.head()

,start_station_name,start_station_id,0
0,1 Ave & E 110 St,7522.02,315
1,1 Ave & E 110 St,7522.02,1265
2,1 Ave & E 16 St,5779.08,678
3,1 Ave & E 16 St,5779.08,3128
4,1 Ave & E 18 St,5854.09,945


In [16]:
df.loc[df["start_station_name"]=="1 Ave & E 110 St", "start_station_id"].apply(repr).unique()


array(["'7522.02'", '7522.02'], dtype=object)

In [17]:
# Convert to string
df["start_station_id"] = df["start_station_id"].astype("string")

# Strip whitespace
df["start_station_id"] = df["start_station_id"].str.strip()

# Remove leading/trailing single or double quotes
df["start_station_id"] = df["start_station_id"].str.strip("'\"")


In [18]:
mapping = (
    df.groupby(["start_station_name", "start_station_id"])
      .size()
      .reset_index(name="count")
)

In [19]:
mapping.head()

,start_station_name,start_station_id,count
0,1 Ave & E 110 St,7522.02,1580
1,1 Ave & E 16 St,5779.08,3806
2,1 Ave & E 18 St,5854.09,4694
3,1 Ave & E 30 St,6079.03,3331
4,1 Ave & E 39 St,6303.01,4803


In [20]:
# how many IDs per name?
ids_per_name = (
    df.groupby("start_station_name")["start_station_id"]
      .nunique()
      .sort_values(ascending=False)
)

ids_per_name.head(20)          # inspect worst offenders
(ids_per_name > 1).sum()       # how many station names have >1 ID?
ids_per_name.value_counts()    # distribution: 1 id, 2 ids, 3 ids, ...


start_station_id
1    2045
2      89
Name: count, dtype: int64

In [21]:
stations = (
    df.dropna(subset=["start_station_name"])
      .groupby("start_station_name")[["start_lat", "start_lng"]]
      .median()
      .reset_index()
)

# Optional: same for end stations (should be identical set)
stations_end = (
    df.dropna(subset=["end_station_name"])
      .groupby("end_station_name")[["end_lat", "end_lng"]]
      .median()
      .reset_index()
)


In [22]:
stations_all = pd.concat([
    stations.rename(columns={"start_lat": "lat", "start_lng": "lng"}),
    stations_end.rename(columns={"end_lat": "lat", "end_lng": "lng"})
], ignore_index=True)

stations_all = (
    stations_all.groupby("start_station_name")  # or end_station_name, same set
                [["lat", "lng"]].median()
                .reset_index()
)


In [23]:
missing_name = df[df["end_station_name"].isna()]


In [25]:
from haversine import haversine

def find_nearest_station(lat, lng, threshold=20):  # 20 meters
    distances = stations_all.apply(
        lambda row: haversine((lat, lng), (row["lat"], row["lng"])),
        axis=1
    )
    min_dist = distances.min()
    return min_dist < threshold  # within 20 meters of a known station

missing_name["matches_station"] = missing_name.apply(
    lambda r: find_nearest_station(r["end_lat"], r["end_lng"]),
    axis=1
)


/var/folders/6c/gmnjxh5x1hn48qy93ptqpz580000gn/T/ipykernel_66597/1040291489.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_name["matches_station"] = missing_name.apply(


In [26]:
missing_name["matches_station"].value_counts()


matches_station
False    6452
True       19
Name: count, dtype: int64